In [202]:
import numpy as np
import math
from numpy import linalg as la
# math.isclose(0.1 + 0.2, 0.3)
np.set_printoptions(suppress=True, linewidth=np.nan)


### Utilities

In [203]:
def build_tableau(A: np.ndarray, b: np.ndarray, c: np.ndarray) -> np.ndarray:
    tableau = np.r_[c.reshape(1, -1), A]
    tableau = np.c_[tableau, np.insert(b, 0, 0)]
    return tableau


In [204]:
def minimum_ratio_test(col: np.ndarray, b: np.ndarray) -> int:
    r_min = 0
    min_val = np.Inf
    for k in range(len(b)):
        if col[k] > 0:
            if min_val > (min_val_temp := b[k] / col[k]):
                min_val = min_val_temp
                r_min = k
    return r_min


In [205]:
def pivoting(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row] = tableau[row]/tableau[row, col]
    # pivot proccess
    for k in range(len(tableau)):
        if k != row:
            tableau[k] = tableau[k] - tableau[k, col]*tableau[row, :]
    return tableau


In [206]:
def correct_negative_cost(tableau: np.ndarray, basic_var: list[int]):
    # correct first row of tableau because the artificial_var and min=-max position is negative
    for index, col in enumerate(basic_var):
        if tableau[0, col] < 0:
            # select the row with 1 because the rest are zeros
            row = index + 1
            tableau[0] = tableau[0] - tableau[0, col] * tableau[row, :]
    return tableau


### Simplex Algorithm

In [207]:
def simplex(tableau: np.ndarray, basic_var: list[int]) -> None:
    print(f"x_B = {basic_var}\n{tableau}")
    while (tableau[0, :-1] > 0).any():
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")


### Big-M
The big-M procedure, each such constraint $i$ is augmented, together with its slack variable, with a so-called artificial variable $u_i$,  and the objective function is augmented with $−Mu_i$, where $M$ is a big positive real number. For big values of $M$ the simplex algorithm will put highest priority on making the value of the factor $Mu_i$ as small as possible, thereby setting the value of $u_i$ equal to zero. Big-M and two-phases are used when $0$ isn't feasible basic solution.

In [208]:
def big_M(tableau: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau for each artificial var because the artificial_var position is -M
    tableau = correct_negative_cost(tableau, basic_var)
    # solve with simplex
    simplex(tableau, basic_var)


### Two-Phases
The two-phase procedure add, in the same manner as employed in the big-M procedure, but instead of adding each artificial variable to the objetive with a large negative coefficient, the objetive function is replace by minus the sum of all artificial variables . During the **first phase**, the simplex algorithm tries to maximize this objetive, effectively trying to give all artificial variables the value zero.  By doing this a feasible basic solution is founded. When this has happened, the **second-phase** starts by replacing the artificial objetive function by the objetive function of the original model and solve this with simplex method.

In [209]:
def two_phases(tableau: np.ndarray, c: np.ndarray, basic_var: list[int], artificial_var: list[int]):
    print(f"x_B = {basic_var}\n{tableau}")
    # correct first row of tableau because the artificial_var position is -1
    tableau = correct_negative_cost(tableau, basic_var)
    print(f"x_B = {basic_var}\n{tableau}")
    
    print("Phase One")
    # while until each artifical var be negative and not be basic var
    while any(((u in basic_var) or (tableau[0, u] > 0)) for u in artificial_var):
        # max criterion
        c_max = np.argmax(tableau[0, :-1])
        # minimum ratio test
        r_min = minimum_ratio_test(col=tableau[1:, c_max], b=tableau[1:, -1]) + 1
        # pivoting
        tableau = pivoting(tableau, r_min, c_max)
        # swap row with col
        basic_var[r_min - 1] = c_max
        print(f"x_B = {basic_var}\n{tableau}")

    print("Phase Two")
    # delete the artificial variable because we found a solution factible
    tableau = np.delete(tableau, artificial_var, axis=1)
    # put c in first row of tableau
    tableau[0, :len(c)] = c
    # solve with simplex
    print(f"x_B = {basic_var}\n{tableau}")
    tableau = correct_negative_cost(tableau, basic_var)
    simplex(tableau, basic_var)


### Examples

**Simplex**: Model Dovetail

In [210]:
A = np.array([[1, 1, 1, 0, 0, 0],
              [3, 1, 0, 1, 0, 0],
              [1, 0, 0, 0, 1, 0],
              [0, 1, 0, 0, 0, 1.0]])

b = np.array([9, 18, 7, 6.])

c = np.array([3, 2, 0, 0, 0, 0.])

tableau = build_tableau(A, b, c)
basic_var = [2, 3, 4, 5]
simplex(tableau, basic_var)


x_B = [2, 3, 4, 5]
[[ 3.  2.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  6.]]
x_B = [2, 0, 4, 5]
[[  0.           1.           0.          -1.           0.           0.         -18.        ]
 [  0.           0.66666667   1.          -0.33333333   0.           0.           3.        ]
 [  1.           0.33333333   0.           0.33333333   0.           0.           6.        ]
 [  0.          -0.33333333   0.          -0.33333333   1.           0.           1.        ]
 [  0.           1.           0.           0.           0.           1.           6.        ]]
x_B = [1, 0, 4, 5]
[[  0.    0.   -1.5  -0.5   0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    1.5]]


**BIG-M**:  Ejemplo visto el ciclo pasado Lineal 6 

In [211]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1", 6: "u2"}
A = np.array([[2, -1, -1, 0, 0, 1, 0],
              [-1, 2, 0, -1, 0, 0, 1],
              [1, 1, 0, 0, 1, 0, 0.]])

b = np.array([4, 2, 12.])
M = 10
c = np.array([2, 1, 0, 0, 0, -M, -M])

tableau = build_tableau(A, b, c)

basic_var = [4, 5, 6]
artificial_var = [5, 6]
big_M(tableau, basic_var, artificial_var)  # FIN NO ACOTADO


x_B = [4, 5, 6]
[[  2.   1.   0.   0.   0. -10. -10.   0.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [4, 5, 6]
[[ -8.  21.   0. -10.   0. -10.   0.  20.]
 [  2.  -1.  -1.   0.   0.   1.   0.   4.]
 [ -1.   2.   0.  -1.   0.   0.   1.   2.]
 [  1.   1.   0.   0.   1.   0.   0.  12.]]
x_B = [4, 1, 6]
[[  2.5   0.    0.    0.5   0.  -10.  -10.5  -1. ]
 [  1.5   0.   -1.   -0.5   0.    1.    0.5   5. ]
 [ -0.5   1.    0.   -0.5   0.    0.    0.5   1. ]
 [  1.5   0.    0.    0.5   1.    0.   -0.5  11. ]]
x_B = [0, 1, 6]
[[  0.           0.           1.66666667   1.33333333   0.         -11.66666667 -11.33333333  -9.33333333]
 [  1.           0.          -0.66666667  -0.33333333   0.           0.66666667   0.33333333   3.33333333]
 [  0.           1.          -0.33333333  -0.66666667   0.           0.33333333   0.66666667   2.66666667]
 [  0.           0.           1.           1.           1.      

**Two-Phases**: Dovetail Model modified

In [212]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "x6", 6: "x7", 7: "u1"}

tableau = np.array([[0, 0, 0, 0, 0, 0, 0, -1, 0],
                    [1, 1, 1, 0, 0, 0, 0, 0, 9.],
                    [3, 1, 0, 1, 0, 0, 0, 0, 18],
                    [1, 0, 0, 0, 1, 0, 0, 0, 7.],
                    [0, 1, 0, 0, 0, 1, 0, 0, 6.],
                    [1, 1, 0, 0, 0, 0, -1, 1, 5]])

c = np.array([3, 2])

basic_var = [2, 3, 4, 5, 7]

artificial_var = [7]

two_phases(tableau, c, basic_var, artificial_var) 


x_B = [2, 3, 4, 5, 7]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
x_B = [2, 3, 4, 5, 7]
[[ 1.  1.  0.  0.  0.  0. -1.  0.  5.]
 [ 1.  1.  1.  0.  0.  0.  0.  0.  9.]
 [ 3.  1.  0.  1.  0.  0.  0.  0. 18.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  7.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Phase One
x_B = [2, 3, 4, 5, 0]
[[ 0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  1.  0.  0.  0.  1. -1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3. -3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1. -1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  0.  6.]
 [ 1.  1.  0.  0.  0.  0. -1.  1.  5.]]
Phase Two
x_B = [2, 3, 4, 5, 0]
[[ 3.  2.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  4.]
 [ 0. -2.  0.  1.  0.  0.  3.  3.]
 [ 0. -1.  0.  0.  1.  0.  1.  2.]
 [ 0.  1.  0.  0.  0.  1.  0.  6.]
 [ 1.  1.  0

### Curso: Programación Lineal y flujo en redes: Cuarta Dirigida

2. Hallar una solución aplicando las tablas del método simplex

In [213]:
A = np.array([[2, 1, 1, 0, 0],
              [1, 0, 0, 1, 0],
              [0, 1, 0, 0, 1.]])

b = np.array([9, 4, 3.])

c = np.array([4, 5, 0, 0, 0.])

basic_var = [2, 3, 4]
tableau = build_tableau(A, b, c)
simplex(tableau, basic_var)

x_B = [2, 3, 4]
[[4. 5. 0. 0. 0. 0.]
 [2. 1. 1. 0. 0. 9.]
 [1. 0. 0. 1. 0. 4.]
 [0. 1. 0. 0. 1. 3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.  -5. -15.]
 [  2.   0.   1.   0.  -1.   6.]
 [  1.   0.   0.   1.   0.   4.]
 [  0.   1.   0.   0.   1.   3.]]
x_B = [0, 3, 1]
[[  0.    0.   -2.    0.   -3.  -27. ]
 [  1.    0.    0.5   0.   -0.5   3. ]
 [  0.    0.   -0.5   1.    0.5   1. ]
 [  0.    1.    0.    0.    1.    3. ]]


Usando 2 fases

In [214]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1"}
A = np.array([[1., 1., 1, 0, 0, 0],
              [-2, -1, 0, 1, 0, 0],
              [0., 1., 0, 0, -1, 1]])

b = np.array([9, 4, 3.])
c_artificial = np.array([0., 0., 0, 0, 0, -1])

tableau = build_tableau(A, b, c_artificial)

c = np.array([4, 5])

basic_var = [2, 3, 5]
artificial_var = [5]
two_phases(tableau, c, basic_var, artificial_var)


x_B = [2, 3, 5]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
x_B = [2, 3, 5]
[[ 0.  1.  0.  0. -1.  0.  3.]
 [ 1.  1.  1.  0.  0.  0.  9.]
 [-2. -1.  0.  1.  0.  0.  4.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Phase One
x_B = [2, 3, 1]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  0.  1.  0.  1. -1.  6.]
 [-2.  0.  0.  1. -1.  1.  7.]
 [ 0.  1.  0.  0. -1.  1.  3.]]
Phase Two
x_B = [2, 3, 1]
[[ 4.  5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  6.]
 [-2.  0.  0.  1. -1.  7.]
 [ 0.  1.  0.  0. -1.  3.]]
x_B = [2, 3, 1]
[[ 4.  5.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  1.  6.]
 [-2.  0.  0.  1. -1.  7.]
 [ 0.  1.  0.  0. -1.  3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.   5. -15.]
 [  1.   0.   1.   0.   1.   6.]
 [ -2.   0.   0.   1.  -1.   7.]
 [  0.   1.   0.   0.  -1.   3.]]
x_B = [4, 3, 1]
[[ -1.   0.  -5.   0.   0. -45.]
 [  1.   0.   1.   0.   1.   6.]
 [ -1.   0.   1.   1.   0.  13.]
 [  1.   1.   1.   0.   0.   9.]]


Usando Big-M

In [215]:
A = np.array([[1., 1., 1, 0, 0, 0],
              [-2, -1, 0, 1, 0, 0],
              [0., 1., 0, 0, -1, 1]])

b = np.array([9, 4, 3.])
M = 10
c = np.array([4, 5, 0, 0, 0, -M])

tableau = build_tableau(A, b, c)

basic_var = [2, 3, 5]
artificial_var = [5]
big_M(tableau, basic_var, artificial_var)  # FIN NO ACOTADO


x_B = [2, 3, 5]
[[  4.   5.   0.   0.   0. -10.   0.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [2, 3, 5]
[[  4.  15.   0.   0. -10.   0.  30.]
 [  1.   1.   1.   0.   0.   0.   9.]
 [ -2.  -1.   0.   1.   0.   0.   4.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [2, 3, 1]
[[  4.   0.   0.   0.   5. -15. -15.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -2.   0.   0.   1.  -1.   1.   7.]
 [  0.   1.   0.   0.  -1.   1.   3.]]
x_B = [4, 3, 1]
[[ -1.   0.  -5.   0.   0. -10. -45.]
 [  1.   0.   1.   0.   1.  -1.   6.]
 [ -1.   0.   1.   1.   0.   0.  13.]
 [  1.   1.   1.   0.   0.   0.   9.]]


3. Determine el gráfico de adyacencia del siguiente modelos LO

In [216]:
from itertools import combinations

In [217]:
A = np.array([[-2, 6, 1, 0, 0],
              [2., 4, 0, 1, 0],
              [1., 0, 0, 0, 1]])

b = np.array([0, 10, 4.])

c = np.array([-1.0, 1, 0, 0, 0])

m, n_plus_m = A.shape

for basic in combinations(range(n_plus_m), m):
    print("Node =", basic)

Node = (0, 1, 2)
Node = (0, 1, 3)
Node = (0, 1, 4)
Node = (0, 2, 3)
Node = (0, 2, 4)
Node = (0, 3, 4)
Node = (1, 2, 3)
Node = (1, 2, 4)
Node = (1, 3, 4)
Node = (2, 3, 4)


6. Resolver por el método de dos faces el siguiente LO

In [218]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "u1", 5: "u2"}
A = np.array([[4., 2, -1, 0, 1, 0],
              [1., 4., 0, -1, 0, 1]])

b = np.array([12, 6.0])
c_artificial = np.array([0., 0., 0, 0, -1, -1])

tableau = build_tableau(A, b, c_artificial)

c = -np.array([2, 3., 0, 0, 0])

basic_var = [4, 5]
artificial_var = [4, 5]
two_phases(tableau, c, basic_var, artificial_var)


x_B = [4, 5]
[[ 0.  0.  0.  0. -1. -1.  0.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
x_B = [4, 5]
[[ 5.  6. -1. -1.  0.  0. 18.]
 [ 4.  2. -1.  0.  1.  0. 12.]
 [ 1.  4.  0. -1.  0.  1.  6.]]
Phase One
x_B = [4, 1]
[[ 3.5   0.   -1.    0.5   0.   -1.5   9.  ]
 [ 3.5   0.   -1.    0.5   1.   -0.5   9.  ]
 [ 0.25  1.    0.   -0.25  0.    0.25  1.5 ]]
x_B = [0, 1]
[[ 0.          0.          0.          0.         -1.         -1.          0.        ]
 [ 1.          0.         -0.28571429  0.14285714  0.28571429 -0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429 -0.07142857  0.28571429  0.85714286]]
Phase Two
x_B = [0, 1]
[[-2.         -3.         -0.         -0.         -0.        ]
 [ 1.          0.         -0.28571429  0.14285714  2.57142857]
 [ 0.          1.          0.07142857 -0.28571429  0.85714286]]
x_B = [0, 1]
[[ 0.          0.         -0.35714286 -0.57142857  7.71428571]
 [ 1.          0.         -0.28571429  0.14285714  2.57142857]


In [219]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "u1"}
A = np.array([[1., 1, 1, 0., 0],
              [-1, 2, 0, -1, 1]])

b = np.array([10, 2.0])
c_artificial = np.array([0., 0., 0, 0, -1])

tableau = build_tableau(A, b, c_artificial)

c = np.array([2, 2., 0, 0])

basic_var = [2, 4]
artificial_var = [4]
two_phases(tableau, c, basic_var, artificial_var)


x_B = [2, 4]
[[ 0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
x_B = [2, 4]
[[-1.  2.  0. -1.  0.  2.]
 [ 1.  1.  1.  0.  0. 10.]
 [-1.  2.  0. -1.  1.  2.]]
Phase One
x_B = [2, 1]
[[ 0.   0.   0.   0.  -1.   0. ]
 [ 1.5  0.   1.   0.5 -0.5  9. ]
 [-0.5  1.   0.  -0.5  0.5  1. ]]
Phase Two
x_B = [2, 1]
[[ 2.   2.   0.   0.   0. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
x_B = [2, 1]
[[ 2.   2.   0.   0.   0. ]
 [ 1.5  0.   1.   0.5  9. ]
 [-0.5  1.   0.  -0.5  1. ]]
x_B = [0, 1]
[[  0.           2.          -1.33333333  -0.66666667 -12.        ]
 [  1.           0.           0.66666667   0.33333333   6.        ]
 [  0.           1.           0.33333333  -0.33333333   4.        ]]
x_B = [0, 1]
[[  0.           0.          -2.           0.         -20.        ]
 [  1.           0.           0.66666667   0.33333333   6.        ]
 [  0.           1.           0.33333333  -0.33333333   4.        ]]
x_B = [3, 1]
[[ -0.   0.  -2.   0. -20.]
 [  

9. Resolver por el método de dos faces el siguiente LO

In [220]:
variable_map = {0: "x1", 1: "x2", 2: "x3", 3: "x4", 4: "x5", 5: "u1"}
A = np.array([[1, 1, 1, 1., 0, 0.],
              [4, 1, 6, 0, -1, 1]])

b = np.array([1, 2.])
c_artificial = np.array([0., 0., 0, 0, 0, -1])

tableau = build_tableau(A, b, c_artificial)

c = np.array([3, 3., 0, 0, 0.0])

basic_var = [2, 4]
artificial_var = [4]
two_phases(tableau, c, basic_var, artificial_var)


x_B = [2, 4]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
x_B = [2, 4]
[[ 0.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  1.  1.  0.  0.  1.]
 [ 4.  1.  6.  0. -1.  1.  2.]]
Phase One
x_B = [2, 0]
[[ 0.    0.    0.    0.    0.   -1.    0.  ]
 [ 0.    0.75 -0.5   1.    0.25 -0.25  0.5 ]
 [ 1.    0.25  1.5   0.   -0.25  0.25  0.5 ]]
Phase Two
x_B = [2, 0]
[[ 3.    3.    0.    0.    0.    0.  ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [2, 0]
[[ 3.    3.    0.    0.    0.    0.  ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [2, 0]
[[ 0.    2.25 -4.5   0.   -0.75 -1.5 ]
 [ 0.    0.75 -0.5   1.   -0.25  0.5 ]
 [ 1.    0.25  1.5   0.    0.25  0.5 ]]
x_B = [1, 0]
[[ 0.          0.         -3.         -3.          0.         -3.        ]
 [ 0.          1.         -0.66666667  1.33333333 -0.33333333  0.66666667]
 [ 1.          0.          1.66666667 -0.33333333  0.33333333

11. Elaborar un programa que al ingresar una $4 \times 6$ matriz arbitraria, verifique si dicha matriz representa las entradas de una tabla simplex (con 2 variables no básicas y 3 variables básicas). Testearlo con una matriz que si represente una tabla simplex y luego con una matrix de igual entradas que el anterior a excepción de la entrada (1, 6).

In [227]:
def check_tableau(tableau: np.ndarray, basic_var: list[int]):
    for index, col in enumerate(basic_var):
        # cost of basic variable is 0.0
        if not tableau[0, col] == 0.0:
            print("Basic cost isn't zero")
            return False
        else:
            for row in range(1, len(basic_var) + 1):
                if row != index + 1:
                    if not 0.0 == tableau[row, col]:
                        print(f"Basic column {col} in position {row} isn't zero")
                        return False
                else:
                    if not tableau[row, col] == 1.0:
                        print(f"Basic column {col} in position {row} isn't one")
                        return False
        # b positive
        if not tableau[index, -1] >= 0.0:
            print("B^{-1} b is negative")
            return False
    print("If it is a simplex table.")
    return True


In [228]:
tableau = np.array([[3., 2., 0., 0., 0., 0., 0.],
                    [1., 1., 1., 0., 0., 0., 9.],
                    [3., 1., 0., 2., 0., 0., -18.],
                    [1., 0., 0., 0., 2., 0., 7.],
                    [0., 1., 0., 0., 0., 1., 6.]])
                    
basic_var = [2, 3, 4, 5]
check_tableau(tableau, basic_var);

Basic column 3 in position 2 isn't one
